In [2]:
%load_ext autoreload
%autoreload 2
%aimport theme
import pandas as pd
import altair as alt
import pygwalker as pyg
from altair import datum
from theme import apply_theme
alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
COLORS = {
    'd': '#56B4E9',
    'j': '#CC79A7',
    'g': '#009E73'
}
GOV_EFR = 0.015346

In [4]:
df = pd.read_csv('../input/Nov-21-2023/3-23-2024_failure_rate_meta.csv')
df

,web_type,continent,country,publisher,fail_rate_meta,se,ci_95L,ci_95U,total_violations,total_checks,units
0,journal,ALL,ALL,ALL,0.027521,0.008960,0.027055,0.027995,351835,8137214,10844
1,journal,NaN,Argentina,ALL,0.042652,0.146270,0.032365,0.056019,1249,27632,34
2,journal,NaN,Australia,ALL,0.026484,0.121066,0.021007,0.033340,1063,40320,64
3,journal,NaN,Austria,ALL,0.014691,0.283566,0.008480,0.025334,629,38776,30
4,journal,NaN,Bangladesh,ALL,0.007832,0.314320,0.004245,0.014406,25,3589,5
...,...,...,...,...,...,...,...,...,...,...,...
542,data_portal,Asia,NaN,NaN,0.085963,0.024360,0.082285,0.089789,47770,778743,1576
543,data_portal,Europe,NaN,NaN,0.068119,0.027672,0.064756,0.071644,56264,971306,1391
544,data_portal,North America,NaN,NaN,0.056968,0.024886,0.054404,0.059646,56266,1173001,1367
545,data_portal,Oceania,NaN,NaN,0.058888,0.086698,0.050147,0.069041,1780,38824,86


In [5]:
df_overall = df[
    (df.continent == 'ALL') &
    (df.country == 'ALL') &
    (df.publisher == 'ALL') |
    (df.web_type == 'government') &
    (df.publisher == 'ALL')
]

df_overall.web_type = df_overall.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')

df_overall

/var/folders/gt/f9y6f2l93wj459z7zbwd4gc80000gq/T/ipykernel_21626/2061737770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_overall.web_type = df_overall.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')


,web_type,continent,country,publisher,fail_rate_meta,se,ci_95L,ci_95U,total_violations,total_checks,units
0,Journal Websites,ALL,ALL,ALL,0.027521,0.008960,0.027055,0.027995,351835,8137214,10844
246,US Government Websites,North America,United States,ALL,0.015346,0.039455,0.014220,0.016559,10829,675955,10844
302,Data Portals,ALL,ALL,ALL,0.070072,0.014573,0.068234,0.071956,10829,675955,10844


In [11]:
base = alt.Chart(df_overall).mark_circle(
    size=60,
    opacity=1
).encode(
    alt.Y('web_type:N', title=None, sort=['US Government Websites', 'Journal Websites', 'Data Portals']),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None)
).properties(
    height=200,
    width=300
)

error = base.mark_errorbar(
    thickness=2,
    color='black'
).encode(
    alt.X('ci_95L', title='Estimated failure rate'),
    alt.X2('ci_95U'),
    color=alt.value('black')
)

plot = base + error + base

plot = apply_theme(plot)

plot

alt.LayerChart(...)

In [9]:
df_journal_continent = df[(df.web_type != 'government') & (df.country != 'ALL') & (df.country.notnull())]

In [21]:
countries_data_portals = [
    'Spain', 'China', 'Germany', 'India', 'Italy', 'Japan', 'Canada', 'France', 'United States', 'United Kingdom'
]
countries_journals = [
    'China', 'Germany', 'India', 'Spain', 'Italy', 'Switzerland', 'Netherlands', 'France', 'United States', 'United Kingdom'
]
df_journal_continent_filtered = df_journal_continent[
    (df_journal_continent.web_type == 'data_portal') & (df_journal_continent.country.isin(countries_data_portals))|
    (df_journal_continent.web_type == 'journal') & (df_journal_continent.country.isin(countries_journals))
]

In [39]:
df_journal_continent_filtered.web_type = df_journal_continent_filtered.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')

/var/folders/gt/f9y6f2l93wj459z7zbwd4gc80000gq/T/ipykernel_14505/2882568888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_journal_continent_filtered.web_type = df_journal_continent_filtered.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')


In [84]:
base = alt.Chart(df_journal_continent_filtered).mark_circle(size=100, opacity=1).encode(
    alt.Y('country:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot

alt.HConcatChart(...)

In [5]:
df_continents = df[(df.web_type != 'government') & (df.continent != 'ALL') & (df.continent.notnull())]
df_continents.web_type = df_continents.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')
df_continents

/var/folders/gt/f9y6f2l93wj459z7zbwd4gc80000gq/T/ipykernel_62316/4148584061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_continents.web_type = df_continents.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')


,web_type,continent,country,publisher,fail_rate_meta,se,ci_95L,ci_95U,total_violations,total_checks,units
67,Journal Websites,Africa,NaN,ALL,0.042936,0.053735,0.038810,0.047478,6258,141402,136
68,Journal Websites,Asia,NaN,ALL,0.047866,0.020626,0.046057,0.049742,100270,1509594,1624
69,Journal Websites,Europe,NaN,ALL,0.022619,0.013266,0.022052,0.023201,140831,4043227,5504
70,Journal Websites,North America,NaN,ALL,0.026146,0.015190,0.025398,0.026915,85158,2038112,3116
71,Journal Websites,Oceania,NaN,ALL,0.040005,0.054172,0.036121,0.044288,5196,127500,156
72,Journal Websites,South America,NaN,ALL,0.040225,0.046357,0.036860,0.043883,14122,277379,308
541,Data Portals,Africa,NaN,NaN,0.071297,0.166639,0.052474,0.096187,586,11410,25
542,Data Portals,Asia,NaN,NaN,0.085963,0.024360,0.082285,0.089789,47770,778743,1576
543,Data Portals,Europe,NaN,NaN,0.068119,0.027672,0.064756,0.071644,56264,971306,1391
544,Data Portals,North America,NaN,NaN,0.056968,0.024886,0.054404,0.059646,56266,1173001,1367


In [6]:
base = alt.Chart(df_continents).mark_circle(size=100, opacity=1).encode(
    alt.Y('continent:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot

alt.HConcatChart(...)

In [4]:
df_org = df[(df.web_type != 'government') & (df.publisher != 'ALL') & (df.publisher.notnull())]
df_org.web_type = df_org.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')
df_org

/var/folders/gt/f9y6f2l93wj459z7zbwd4gc80000gq/T/ipykernel_19590/3082492987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_org.web_type = df_org.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')


,web_type,continent,country,publisher,fail_rate_meta,se,ci_95L,ci_95U,total_violations,total_checks,units
73,Journal Websites,NaN,NaN,ABV-press Publishing house,0.237028,0.029955,0.226575,0.247808,1461,6166,6
74,Journal Websites,NaN,NaN,Academic Press,0.018577,0.049576,0.016886,0.020434,415,23741,107
75,Journal Websites,NaN,NaN,Adis Journals,0.006666,0.086677,0.005631,0.007891,134,20126,19
76,Journal Websites,NaN,NaN,Agricultural Research Communication Centre,0.025250,0.085604,0.021434,0.029726,140,5547,5
77,Journal Websites,NaN,NaN,Akademiai Kiado,0.071894,0.070333,0.063222,0.081653,1130,15886,13
...,...,...,...,...,...,...,...,...,...,...,...
499,Data Portals,NaN,NaN,Xiamen University,0.141697,0.332546,0.079216,0.240583,151,2735,10
500,Data Portals,NaN,NaN,Yale University,0.049029,0.278787,0.028987,0.081760,317,6708,13
501,Data Portals,NaN,NaN,Yonsei University,0.069388,0.419297,0.031739,0.145005,137,5007,10
502,Data Portals,NaN,NaN,Zhejiang University,0.092053,0.159748,0.069015,0.121776,1217,28571,45


In [10]:
top_10_journal_publishers_by_size = df_org[(df_org.web_type == 'Journal Websites') & (df_org.units > 150)]

In [20]:
top_10_data_portal_publishers_by_size = df_org[(df_org.web_type == 'Data Portals') & (df_org.units > 27)]

In [23]:
df_org_filtered = pd.concat([top_10_journal_publishers_by_size, top_10_data_portal_publishers_by_size])
df_org_filtered

,web_type,continent,country,publisher,fail_rate_meta,se,ci_95L,ci_95U,total_violations,total_checks,units
94,Journal Websites,NaN,NaN,BioMed Central Ltd.,0.013743,0.022781,0.013151,0.014362,3271,239865,243
124,Journal Websites,NaN,NaN,Elsevier,0.020047,0.023184,0.019173,0.020959,6860,279712,1135
153,Journal Websites,NaN,NaN,John Wiley and Sons,0.018094,0.069660,0.015821,0.020686,1540,68170,246
164,Journal Websites,NaN,NaN,Lippincott Williams and Wilkins,0.070063,0.017993,0.067799,0.072395,32711,441550,280
181,Journal Websites,NaN,NaN,Oxford University Press,0.065564,0.049569,0.059857,0.071773,9053,225362,199
197,Journal Websites,NaN,NaN,Routledge,0.016935,0.050866,0.015353,0.018677,394,24561,176
202,Journal Websites,NaN,NaN,SAGE Publications,0.018545,0.031000,0.017471,0.019684,1062,60849,370
211,Journal Websites,NaN,NaN,Springer,0.007759,0.034527,0.007255,0.008298,7706,860822,826
215,Journal Websites,NaN,NaN,Taylor and Francis Ltd.,0.017495,0.030024,0.016511,0.018535,1132,69139,442
239,Journal Websites,NaN,NaN,Wiley-Blackwell,0.017155,0.035221,0.016029,0.018358,1477,71639,519


In [24]:
base = alt.Chart(df_org_filtered).mark_circle(size=100, opacity=1).encode(
    alt.Y('publisher:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot

alt.HConcatChart(...)